In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
import pandas as pd
train=pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
test=pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')

train=reduce_memory_usage(train)
test=reduce_memory_usage(test)

import gc
gc.collect()

x_train=train.drop(['id','target'],axis=1)
x_test=test.drop(['id'],axis=1)

y_data=train.target



In [ ]:
x_train,x_val,y_train,y_test=train_test_split(x_train,y_data,test_size=0.3)

In [ ]:
model=XGBClassifier(tree_method='gpu_hist',predictor='gpu_predictor')
model.fit(x_train,y_train)

In [ ]:
y_pred=model.predict_proba(x_val)[:,1]

In [ ]:
acc=roc_auc_score(y_test,y_pred)

In [ ]:
print(acc)

In [ ]:
y_test=model.predict_proba(x_test)[:,1]

In [ ]:
submission=pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
submission.target=y_test
submission.to_csv('submission.csv',index=False)